# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv("../output_data/cities.csv")
weather_df.head(5)

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Okhotsk,59.3833,143.3000,-9.11,80,0,18.03,RU,1612055798
1,1,Portland,45.5234,-122.6762,48.99,81,90,12.66,US,1612055602
2,2,Port Alfred,-33.5906,26.8910,63.00,92,0,0.89,ZA,1612056001
3,3,Vaini,-21.2000,-175.2000,86.00,79,75,13.80,TO,1612055978
4,4,Butaritari,3.0707,172.7902,81.32,78,30,18.86,KI,1612055542


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configuring gmaps with API key
gmaps.configure(api_key=g_key)

# Storing lat and lng into locations 
locations = weather_df[["Lat", "Lng"]].astype(float)

# Stoing humidity as the weight

weight = weather_df["Humidity"].astype(float)

In [6]:
# Plot heat map
figure = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(weight)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weight, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
figure.add_layer(heat_layer)

# Display figure
figure

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
# Filtering dataframe with following criteria: wind <10mph, temperature >70 and <80, no cloudiness and drop any cities that don't meet these conditions
filtered_df = weather_df.loc[(weather_df["Wind Speed"] < 10) & (weather_df["Cloudiness"] == 0) & \
                                   (weather_df["Max Temp"] > 70) & (weather_df["Max Temp"] < 80)].dropna()

filtered_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
75,75,Robertsport,6.7533,-11.3686,79.99,86,0,9.06,LR,1612056367
141,141,Saint-Philippe,-21.3585,55.7679,78.80,65,0,6.91,RE,1612055790
190,190,Ahipara,-35.1667,173.1667,72.28,49,0,5.99,NZ,1612055783
232,232,Yenagoa,4.9247,6.2642,78.08,87,0,3.06,NG,1612056014
344,344,Beïnamar,8.6698,15.3813,71.33,22,0,3.94,TD,1612056390
347,347,Srīvardhan,18.0333,73.0167,73.04,45,0,5.48,IN,1612056390
351,351,Ambilobe,-13.2000,49.0500,72.61,93,0,2.53,MG,1612055799
383,383,Jamame,0.0722,42.7506,71.69,89,0,7.81,SO,1612056393
429,429,Kukuna,9.4000,-12.6667,71.26,81,0,3.09,SL,1612056397
453,453,Inhambane,-23.8650,35.3833,78.12,80,0,5.64,MZ,1612056399


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
# Storing into new dataframe, dropping unneeded columns, adding requested column and displaying new dataframe
hotel_df = filtered_df.loc[:,["City", "Country", "Lat", "Lng"]]
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
75,Robertsport,LR,6.7533,-11.3686,
141,Saint-Philippe,RE,-21.3585,55.7679,
190,Ahipara,NZ,-35.1667,173.1667,
232,Yenagoa,NG,4.9247,6.2642,
344,Beïnamar,TD,8.6698,15.3813,
347,Srīvardhan,IN,18.0333,73.0167,
351,Ambilobe,MG,-13.2000,49.0500,
383,Jamame,SO,0.0722,42.7506,
429,Kukuna,SL,9.4000,-12.6667,
453,Inhambane,MZ,-23.8650,35.3833,


In [9]:
# Storing base URL for API search
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Creating parameters dictionary by which hotels will be searched
parameters = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

for index, row in hotel_df.iterrows():
    # Getting city name, lat, lnt from dataframe
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # Adding keyword to paramaters dictionary
    parameters["location"] = f"{lat},{lng}"

    # Creating URL and making API response
    print(f"Retrieving Result Number {index}: {city_name}.")
    response = requests.get(base_url, parameters).json()
    
    # Extracting results
    results = response['results']
    
    # Adding hotel names to dataframe
    try:
        print(f"Nearest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # Skip errors (no hotel found)
    except (KeyError, IndexError):
        print("No result.")
        
    print("------------")

print("-------End of Search-------")

Retrieving Result Number 75: Robertsport.
Nearest hotel in Robertsport is Nana's Lodge.
------------
Retrieving Result Number 141: Saint-Philippe.
Nearest hotel in Saint-Philippe is Les Embruns Du Baril.
------------
Retrieving Result Number 190: Ahipara.
Nearest hotel in Ahipara is Ahipara Bay Motel.
------------
Retrieving Result Number 232: Yenagoa.
Nearest hotel in Yenagoa is De Brass Suites.
------------
Retrieving Result Number 344: Beïnamar.
No result.
------------
Retrieving Result Number 347: Srīvardhan.
Nearest hotel in Srīvardhan is Tranquil Beach Resort.
------------
Retrieving Result Number 351: Ambilobe.
Nearest hotel in Ambilobe is Hotel National.
------------
Retrieving Result Number 383: Jamame.
No result.
------------
Retrieving Result Number 429: Kukuna.
No result.
------------
Retrieving Result Number 453: Inhambane.
Nearest hotel in Inhambane is Sentidos Beach Retreat.
------------
Retrieving Result Number 455: Numan.
Nearest hotel in Numan is Numan Hotel.
--------

In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Display figure
figure.add_layer(markers)

figure

Figure(layout=FigureLayout(height='420px'))